# Capped 3-mer backbone 2-D TorsionDrives

Prepare 2-D TorsionDrives on up to the backbone dihedrals phi (C-N-CA-C) and psi (N-CA-C-N) for the central residue in capped 3-mers (Ace-X-Y-X-Nme), where the flanking residue X is Ala or Val and the central residue Y is one of the 26 canonical amino acids including common protomers and tautomers. Each molecule will be prepared with the sidechain dihedrals of the central residue and Val of the flanking residues set to values corresponding to the most populated sidechain rotamer. For the TorsionDrive, up to the first two sidechain dihedrals of the central residue but not the flanking residues will be constrained to these values. For proline, perform a scan with the peptide bond (omega) in both the *cis* and *trans* conformations.

In [1]:
import json
import numpy
from openeye import oechem, oeomega
from openff.qcsubmit.datasets import TorsiondriveDataset
from openff.qcsubmit.factories import TorsiondriveDatasetFactory
from openff.qcsubmit.workflow_components import TorsionIndexer
from openff.toolkit.topology import Molecule
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.utils import GLOBAL_TOOLKIT_REGISTRY
from openmm import openmm, unit
import os
from qcelemental.molutil import guess_connectivity
import qcportal

/home/ccavende/anaconda3/envs/openff-qca/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ccavende/anaconda3/envs/openff-qca/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ccavende/anaconda3/envs/openff-qca/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ccavende/anaconda3/envs/openff-qca/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
rad_to_deg = 180 / numpy.pi

# Central residues in capped 3-mers, i.e. X in Ace-Y-X-Y-Nme
residue_names = [
    'ALA', 'ARG', 'ASH', 'ASN', 'ASP', 'CYS', 'CYX', 'GLH', 'GLN', 'GLU', 'GLY', 'HID', 'HIE', 'HIP',
    'ILE', 'LEU', 'LYN', 'LYS', 'MET', 'PHE', 'cisPRO', 'transPRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'
]

# Flanking residues in capped 3-mers, i.e. Y in Ace-Y-X-Y-Nme
flanking_residues = ['ALA', 'VAL']

# SMILES for amino acid monomers and ACE and NME caps
residue_smiles = {
    'ALA': 'N[C@@H](C)C(=O)',
    'ARG': 'N[C@@H](CCC[NH+]=C(N)N)C(=O)',
    'ASH': 'N[C@@H](CC(=O)O)C(=O)',
    'ASN': 'N[C@@H](CC(=O)N)C(=O)',
    'ASP': 'N[C@@H](CC(=O)[O-])C(=O)',
    'CYS': 'N[C@@H](CS)C(=O)',
    'CYX': 'N[C@@H](CSSC[C@@H](C(=O)NC)NC(=O)C)C(=O)',
    'GLH': 'N[C@@H](CCC(=O)O)C(=O)',
    'GLN': 'N[C@@H](CCC(=O)N)C(=O)',
    'GLU': 'N[C@@H](CCC(=O)[O-])C(=O)',
    'GLY': 'NCC(=O)',
    'HID': 'N[C@@H](CC1=CN=CN1)C(=O)',
    'HIE': 'N[C@@H](CC1=CNC=N1)C(=O)',
    'HIP': 'N[C@@H](CC1=CNC=[NH+]1)C(=O)',
    'ILE': 'N[C@@H]([C@@H](C)CC)C(=O)',
    'LEU': 'N[C@@H](CC(C)C)C(=O)',
    'LYN': 'N[C@@H](CCCCN)C(=O)',
    'LYS': 'N[C@@H](CCCC[NH3+])C(=O)',
    'MET': 'N[C@@H](CCSC)C(=O)',
    'PHE': 'N[C@@H](Cc1ccccc1)C(=O)',
    'cisPRO': 'N1CCC[C@H]1C(=O)',
    'transPRO': 'N1CCC[C@H]1C(=O)',
    'SER': 'N[C@@H](CO)C(=O)',
    'THR': 'N[C@@H]([C@H](O)C)C(=O)',
    'TRP': 'N[C@@H](CC1=CNc2c1cccc2)C(=O)',
    'TYR': 'N[C@@H](Cc1ccc(cc1)O)C(=O)',
    'VAL': 'N[C@@H](C(C)C)C(=O)',
    'ACE': 'CC(=O)',
    'NME': 'NC',
}

# Dihedral SMARTS for central residue in a capped 3-mer Ace-X-Y-Z-Nme
CCONC = '[#6X4]-[#6X3](=O)-[#7X3]-[#6X4]'
dihedral_smarts = {
    res_name: {
        'phi': f'{CCONC}-[#6X3:1](=O)-[#7X3:2]-[#6X4:3]-[#6X3:4](=O)-[#7X3]-{CCONC}',
        'psi': f'{CCONC}-[#6X3](=O)-[#7X3:1]-[#6X4:2]-[#6X3:3](=O)-[#7X3:4]-{CCONC}',
        'chi1': f'{CCONC}-[#6X3](=O)-[#7X3:1]-[#6X4:2](-[#6X3](=O)-[#7X3]-{CCONC})-[#6X4:3]-[!#1:4]',
        'chi2': f'{CCONC}-[#6X3](=O)-[#7X3]-[#6X4:1](-[#6X3](=O)-[#7X3]-{CCONC})-[#6X4:2]-[#6:3]~[!#1:4]',
    } for res_name in residue_names
}
dihedral_smarts['ASH']['chi2'] = f'{CCONC}-[#6X3](=O)-[#7X3]-[#6X4:1](-[#6X3](=O)-[#7X3]-{CCONC})-[#6X4:2]-[#6:3]=[#8:4]'
dihedral_smarts['ASN']['chi2'] = f'{CCONC}-[#6X3](=O)-[#7X3]-[#6X4:1](-[#6X3](=O)-[#7X3]-{CCONC})-[#6X4:2]-[#6:3]=[#8:4]'
dihedral_smarts['HID']['chi2'] = f'{CCONC}-[#6X3](=O)-[#7X3]-[#6X4:1](-[#6X3](=O)-[#7X3]-{CCONC})-[#6X4:2]-[#6:3]~[#7:4]'
dihedral_smarts['HIE']['chi2'] = f'{CCONC}-[#6X3](=O)-[#7X3]-[#6X4:1](-[#6X3](=O)-[#7X3]-{CCONC})-[#6X4:2]-[#6:3]~[#7:4]'
dihedral_smarts['HIP']['chi2'] = f'{CCONC}-[#6X3](=O)-[#7X3]-[#6X4:1](-[#6X3](=O)-[#7X3]-{CCONC})-[#6X4:2]-[#6:3]~[#7:4]'
dihedral_smarts['ILE']['chi1'] = f'{CCONC}-[#6X3](=O)-[#7X3:1]-[#6X4:2](-[#6X3](=O)-[#7X3]-{CCONC})-[#6X4:3]-[#6H2:4]'
dihedral_smarts['cisPRO']['omega'] = (
    f'[#6X4]-[#6X3](=O)-[#7X3]-[#6X4:1]-[#6X3:2](=O)-[#7X3:3]-[#6X4:4]-[#6X3](=O)-[#7X3]-{CCONC}'
)
dihedral_smarts['transPRO']['omega'] = (
    f'[#6X4]-[#6X3](=O)-[#7X3]-[#6X4:1]-[#6X3:2](=O)-[#7X3:3]-[#6X4:4]-[#6X3](=O)-[#7X3]-{CCONC}'
)
dihedral_smarts['THR']['chi1'] = f'{CCONC}-[#6X3](=O)-[#7X3:1]-[#6X4:2](-[#6X3](=O)-[#7X3]-{CCONC})-[#6X4:3]-[#8:4]'
dihedral_smarts['TRP']['chi2'] = f'{CCONC}-[#6X3](=O)-[#7X3]-[#6X4:1](-[#6X3](=O)-[#7X3]-{CCONC})-[#6X4:2]-[#6:3]~[#6R2:4]'

# Number of rotamers to generate for each residue
max_rotamers = {res_name: 1 for res_name in residue_names}

# Map residue name to a valid name for OpenEye's rotamer library
oe_res_name_rotamer_map = {res_name: res_name for res_name in residue_names}
oe_res_name_rotamer_map['ASH'] = 'ASP'
oe_res_name_rotamer_map['CYX'] = 'CYS'
oe_res_name_rotamer_map['GLH'] = 'GLU'
oe_res_name_rotamer_map['HID'] = 'HIS'
oe_res_name_rotamer_map['HIE'] = 'HIS'
oe_res_name_rotamer_map['HIP'] = 'HIS'
oe_res_name_rotamer_map['LYN'] = 'LYS'
oe_res_name_rotamer_map['cisPRO'] = 'PRO'
oe_res_name_rotamer_map['transPRO'] = 'PRO'

# Setup backbone constraint values
backbone_constraints = {
    'alpha': {'phi': -60, 'psi': -45},
    'beta': {'phi': -135, 'psi': 135},
}

# Restrict proline phi to [-180, 60] to avoid ring strain
dihedral_range = {}
dihedral_spacing = {}
for res_name in residue_names:

    dihedral_spacing[res_name] = {'phi': 15, 'psi': 15}

    if res_name == 'cisPRO' or res_name == 'transPRO':

        dihedral_range[res_name] = {
            'phi': numpy.arange(-180, 61, dihedral_spacing[res_name]['phi']),
            'psi': numpy.arange(-180, 180, dihedral_spacing[res_name]['psi']),
        }

    else:

        dihedral_range[res_name] = {
            'phi': numpy.arange(-180, 180, dihedral_spacing[res_name]['phi']),
            'psi': numpy.arange(-180, 180, dihedral_spacing[res_name]['psi']),
        }


# Generate backbone conformations

Use the OpenEye Toolkit to generate sidechain rotamers and scan backbone conformations for Ace-Ala-X-Ala-Nme and Ace-Val-X-Val-Nme for each amino acid. Write an SDF file for each capped 3-mer with backbone dihedral scans as conformers.

In [3]:
# Get an OEAtom from a residue match predicate and an atom name
def get_residue_atom(molecule, residue_predicate, atom_name, residue_id=None):

    if residue_id is not None:
        return molecule.GetAtom(oechem.OEAndAtom(
            oechem.OEAndAtom(oechem.OEAtomMatchResidue(residue_predicate), oechem.OEHasResidueNumber(residue_id)),
            oechem.OEHasAtomName(atom_name)
        ))
    else:
        return molecule.GetAtom(
            oechem.OEAndAtom(oechem.OEAtomMatchResidue(residue_predicate), oechem.OEHasAtomName(atom_name)))

# Get array of rotamers sorted by probability
# res_name is a str corresponding to the 3-letter code in all caps
def get_sorted_rotamers(res_name, rotamer_map = oe_res_name_rotamer_map):

    rotamers = []
    probs = []

    for rot in oechem.OEGetRotamers(oechem.OEGetResidueIndex(rotamer_map[res_name])):
        rotamers.append(rot)
        probs.append(rot.GetProbability())

    # Get indices that would sort probabilities high-to-low
    prob_sort = numpy.argsort(probs)[::-1]

    # Sort array of rotamers by those indices
    sorted_rotamers = numpy.array(rotamers)[prob_sort]

    # Print rotamer probabilities and sidechain dihedrals
    print('#   Prob   Chi 1  Chi 2  Chi 3  Chi 4')
    for i in range(sorted_rotamers.size):

        rot = sorted_rotamers[i]
        print(
            f'{i:2d} {rot.GetProbability():6.2f} {rot.GetChi1():6.1f} {rot.GetChi2():6.1f} {rot.GetChi3():6.1f} '
            f'{rot.GetChi4():6.1f}'
        )

    return sorted_rotamers

# Get list of residues
# polymer is an OEHierView
def get_residues(polymer):

    residues = []

    for chain in polymer.GetChains():
        for fragment in chain.GetFragments():
            for res in fragment.GetResidues():
                residues.append(res)

    return residues

# Get dict of backbone and sidechain dihedrals in deg
# residue is an OEHierResidue or an OEAtom
def get_dihedrals(residue):

    dihedrals = {}
    dihedrals['phi'] = oechem.OEGetPhi(residue) * rad_to_deg
    dihedrals['psi'] = oechem.OEGetPsi(residue) * rad_to_deg
    omega = oechem.OEGetTorsion(residue, 3)
    if omega != -100.0:
        dihedrals['omega'] = omega * rad_to_deg
    for chi_idx in oechem.OEGetChis(residue):
        dihedrals[f'chi{(chi_idx - 3):d}'] = oechem.OEGetTorsion(residue, chi_idx) * rad_to_deg
    return dihedrals

# Print backbone and sidechain dihedrals in deg
# residue is an OEHierResidue or an OEAtom
def print_dihedrals(residue):

    dihedrals = get_dihedrals(residue)
    out = f'{dihedrals["phi"]:8.3f} {dihedrals["psi"]:8.3f}'
    if 'omega' in dihedrals:
        out += f' {dihedrals["omega"]:8.3f}'
    for chi_idx in oechem.OEGetChis(residue):
        out += f' {dihedrals[f"chi{(chi_idx - 3):d}"]:8.3f}'
    print(out)

# In reference molecule, get internal coords of atom rd with respect to atoms ra, rb, and rc. In target molecule,
# set Cartesian coords of atom td using the same internal coords with respect to atoms ta, tb, and tc.
# Reference and target are OE molecules. ra, rb, rc, rd, ta, tb, tc, and td are OE atoms.
# overwrite_angle overwrites the angle between rb, rc, and rd with an angle given in rad.
# overwrite_dihedral overwrites the dihedral between ra, rb, rc, and rd with an angle given in rad.
def copy_internal_coords(
    reference, target, ra, rb, rc, rd, ta, tb, tc, td, overwrite_angle = None, overwrite_dihedral = None):

    distance = oechem.OEGetDistance(reference, rd, rc)

    if overwrite_angle is None:
        angle = oechem.OEGetAngle(reference, rd, rc, rb)
    else:
        angle = overwrite_angle

    if overwrite_dihedral is None:
        dihedral = oechem.OEGetTorsion(reference, rd, rc, rb, ra)
    else:
        dihedral = overwrite_dihedral

    # Vector from c to d in local coordinate system
    dx = distance * numpy.sin(angle) * numpy.sin(dihedral)
    dy = distance * numpy.sin(angle) * numpy.cos(dihedral)
    dz = distance * numpy.cos(angle)

    # Get local coordinate system
    target_coords = target.GetCoords()
    r_cb = numpy.array(target_coords[tb.GetIdx()]) - numpy.array(target_coords[tc.GetIdx()])
    r_ba = numpy.array(target_coords[ta.GetIdx()]) - numpy.array(target_coords[tb.GetIdx()])
    r_cb /= numpy.linalg.norm(r_cb)
    r_ba /= numpy.linalg.norm(r_ba)

    # Get normal vector to plane containing atoms a, b, and c
    ba_cross_cb = numpy.cross(r_ba, r_cb)
    ba_cross_cb /= numpy.linalg.norm(ba_cross_cb)

    # z axis is oriented from c to b
    z = r_cb

    # y axis is orthogonal to z and in plane of atoms a, b, and c
    y = numpy.cross(z, ba_cross_cb)
    y /= numpy.linalg.norm(y)

    # x axis is orthogonal to y and z
    x = numpy.cross(y, z)
    x /= numpy.linalg.norm(x)

    # Update coordinates of atom td
    td_coords = target_coords[tc.GetIdx()] + numpy.dot(numpy.array([x, y, z]).T, numpy.array([dx, dy, dz]))
    target.SetCoords(td, td_coords)


In [7]:
res_name_pred = oechem.OEAtomMatchResidueID()

# Dictionary of dihedrals in deg indexed by residue name, then backbone conformation index,
# then sidechain conformer index, then dihedral name
dihedrals_by_conf_idx = {}

with oechem.oemolostream() as ofs:

    for central_residue in residue_names:

        for flanking_residue in flanking_residues:

            mol_name = f'Ace-{flanking_residue}-{central_residue}-{flanking_residue}-Nme'.title()

            # Concatenate residue SMILES to get molecule SMILES
            mol_smiles = (
                f'{residue_smiles["ACE"]}{residue_smiles[flanking_residue]}{residue_smiles[central_residue]}'
                f'{residue_smiles[flanking_residue]}{residue_smiles["NME"]}'
            )

            print(mol_name, mol_smiles)
            os.makedirs(os.path.join('capped_3-mer_conformations', mol_name), exist_ok = True)

            mol = oechem.OEGraphMol()
            if oechem.OESmilesToMol(mol, mol_smiles):

                dihedrals_by_conf_idx[mol_name] = {}

                # Generate an initial conformer with explicit hydrogens
                oechem.OEAddExplicitHydrogens(mol)
                oeomega.OEConformerBuilder().Build(mol)
                oechem.OECenter(mol)

                # Perceive residues
                oechem.OEPerceiveResidues(mol)

                # Get hierarchy view of molecule and list of sidechain rotamers sorted by probability
                hv_mol = oechem.OEHierView(mol)
                mol_residues = get_residues(hv_mol)
                mol_rotamers = get_sorted_rotamers(central_residue)

                # Get atoms that make up phi and psi
                res_name_pred.SetChainID('B' if central_residue == 'CYX' else 'A')
                res_name_pred.SetName(oe_res_name_rotamer_map[central_residue])
                n = get_residue_atom(mol, res_name_pred, ' N  ', residue_id = 2)
                ca = get_residue_atom(mol, res_name_pred, ' CA ', residue_id = 2)
                c = get_residue_atom(mol, res_name_pred, ' C  ', residue_id = 2)

                res_name_pred.SetName(oe_res_name_rotamer_map[flanking_residue])
                c_prev = get_residue_atom(mol, res_name_pred, ' C  ', residue_id = 1)
                n_next = get_residue_atom(mol, res_name_pred, ' N  ', residue_id = 3)

                # Write conformers for each rotamer to SDF. OESetTorsion fails for rings, so set side chain torsions
                # and phi for proline manually.
                if central_residue == 'cisPRO' or central_residue == 'transPRO':

                    # Get CA of previous residue for omega torsion
                    ca_prev = get_residue_atom(mol, res_name_pred, ' CA ', residue_id = 1)

                    # Get atoms beyond main backbone in proline residue
                    res_name_pred.SetName(oe_res_name_rotamer_map[central_residue])
                    ha = get_residue_atom(mol, res_name_pred, ' HA ', residue_id = 2)
                    cb = get_residue_atom(mol, res_name_pred, ' CB ', residue_id = 2)
                    hb1 = get_residue_atom(mol, res_name_pred, ' HB2', residue_id = 2)
                    hb2 = get_residue_atom(mol, res_name_pred, ' HB3', residue_id = 2)
                    cg = get_residue_atom(mol, res_name_pred, ' CG ', residue_id = 2)
                    hg1 = get_residue_atom(mol, res_name_pred, ' HG2', residue_id = 2)
                    hg2 = get_residue_atom(mol, res_name_pred, ' HG3', residue_id = 2)
                    cd = get_residue_atom(mol, res_name_pred, ' CD ', residue_id = 2)
                    hd1 = get_residue_atom(mol, res_name_pred, ' HD2', residue_id = 2)
                    hd2 = get_residue_atom(mol, res_name_pred, ' HD3', residue_id = 2)
                    o = get_residue_atom(mol, res_name_pred, ' O  ', residue_id = 2)

                    # Reference molecule for reference internal coordinates
                    ref_mol = oechem.OEGraphMol(mol)

                    ref_n = get_residue_atom(ref_mol, res_name_pred, ' N  ', residue_id = 2)
                    ref_ca = get_residue_atom(ref_mol, res_name_pred, ' CA ', residue_id = 2)
                    ref_ha = get_residue_atom(ref_mol, res_name_pred, ' HA ', residue_id = 2)
                    ref_cb = get_residue_atom(ref_mol, res_name_pred, ' CB ', residue_id = 2)
                    ref_hb1 = get_residue_atom(ref_mol, res_name_pred, ' HB2', residue_id = 2)
                    ref_hb2 = get_residue_atom(ref_mol, res_name_pred, ' HB3', residue_id = 2)
                    ref_cg = get_residue_atom(ref_mol, res_name_pred, ' CG ', residue_id = 2)
                    ref_hg1 = get_residue_atom(ref_mol, res_name_pred, ' HG2', residue_id = 2)
                    ref_hg2 = get_residue_atom(ref_mol, res_name_pred, ' HG3', residue_id = 2)
                    ref_cd = get_residue_atom(ref_mol, res_name_pred, ' CD ', residue_id = 2)
                    ref_hd1 = get_residue_atom(ref_mol, res_name_pred, ' HD2', residue_id = 2)
                    ref_hd2 = get_residue_atom(ref_mol, res_name_pred, ' HD3', residue_id = 2)
                    ref_c = get_residue_atom(ref_mol, res_name_pred, ' C  ', residue_id = 2)
                    ref_o = get_residue_atom(ref_mol, res_name_pred, ' O  ', residue_id = 2)

                    res_name_pred.SetName(oe_res_name_rotamer_map[flanking_residue])
                    ref_c_prev = get_residue_atom(ref_mol, res_name_pred, ' C  ', residue_id = 1)
                    ref_n_next = get_residue_atom(ref_mol, res_name_pred, ' N  ', residue_id = 3)

                    ref_cb_cg_distance = oechem.OEGetDistance(ref_mol, ref_cb, ref_cg)
                    ref_cg_cd_distance = oechem.OEGetDistance(ref_mol, ref_cg, ref_cd)
                    ref_cb_cg_dist_sq = ref_cb_cg_distance * ref_cb_cg_distance
                    ref_cg_cd_dist_sq = ref_cg_cd_distance * ref_cg_cd_distance

                    for i in range(max_rotamers[central_residue]):

                        dihedrals_by_conf_idx[mol_name][i] = {}
                        conf_idx = 0

                        # Create copy of OE molecule with no conformers
                        new_mol = oechem.OEMol(mol)
                        new_mol.DeleteConfs()

                        # Set omega to cis or trans
                        omega = 0 if central_residue == 'cisPRO' else 180
                        oechem.OESetTorsion(mol, ca_prev, c_prev, n, ca, omega / rad_to_deg)

                        # Get side chain dihedrals for proline rotamers
                        mol_phi = oechem.OEGetTorsion(mol, c_prev, n, ca, c) * rad_to_deg
                        mol_psi = oechem.OEGetTorsion(mol, n, ca, c, n_next) * rad_to_deg
                        mol_omega = oechem.OEGetTorsion(mol, ca_prev, c_prev, n, ca) * rad_to_deg
                        rot_chi1 = mol_rotamers[i].GetChi1()
                        rot_chi2 = mol_rotamers[i].GetChi2()
                        print(f'{mol_name} rotamer {i:d}')
                        print(f'{mol_phi:8.3f} {mol_psi:8.3f} {mol_omega:8.3f} {rot_chi1:8.3f} {rot_chi2:8.3f}')

                        # Scan backbone dihedrals
                        for phi in dihedral_range[central_residue]['phi']:

                            # Try to find good ring atom positions for given value of phi
                            # Otherwise increment phi towards -60 deg
                            sqrt_arg = -1
                            tmp_phi = phi
                            while sqrt_arg < 0:

                                # Given coordinates of previous residue and proline N and CA, CD is also fixed by its
                                # internal coordinates with respect to N, CA, and the previous residue C.
                                # Set coordinates of proline C using phi and internal coordinates of CA and N.
                                copy_internal_coords(
                                    ref_mol, mol, ref_c_prev, ref_n, ref_ca, ref_c, c_prev, n, ca, c,
                                    overwrite_dihedral = tmp_phi / rad_to_deg
                                )

                                # Set coordinates of CB using internal coordinates of CA, N, and C
                                copy_internal_coords(
                                    ref_mol, mol, ref_c, ref_n, ref_ca, ref_cb, c, n, ca, cb)

                                # Set coordinates of CG using bond distances to CB and CD and the rotamer chi1. To
                                # do this, use the internal coordinates of CD with respect to CB, CA, and N to find
                                # the angle CA-CB-CG that satisfies the CG-CD distance in spherical coordinates
                                # D^2 = RG^2 + RD^2 - 2 RG RD (sin AG sin AD cos(PhiG - PhiD) + cos AG cos AD)
                                # A sin AG + B cos AG + C = 0
                                # A = 2 RG RD sin AD cos(PhiG - PhiD)
                                # B = 2 RG RD cos AD
                                # C = D^2 - RG^2 - RD^2
                                # AG = 2 arctan((A +/- sqrt(A^2 + B^2 - C^2)) / (B - C))
                                cb_cd_distance = oechem.OEGetDistance(mol, cb, cd)
                                ca_cb_cd_angle = oechem.OEGetAngle(mol, ca, cb, cd)
                                n_ca_cb_cd_dihedral = oechem.OEGetTorsion(mol, n, ca, cb, cd)
                                A = 2 * ref_cb_cg_distance * cb_cd_distance * numpy.sin(ca_cb_cd_angle) * numpy.cos(
                                    rot_chi1 / rad_to_deg - n_ca_cb_cd_dihedral)
                                B = 2 * ref_cb_cg_distance * cb_cd_distance * numpy.cos(ca_cb_cd_angle)
                                C = ref_cg_cd_dist_sq - ref_cb_cg_dist_sq - cb_cd_distance * cb_cd_distance
                                sqrt_arg = A * A + B * B - C * C

                                if tmp_phi > -60:
                                    tmp_phi -= dihedral_spacing[central_residue]['phi']
                                else:
                                    tmp_phi += dihedral_spacing[central_residue]['phi']

                            ca_cb_cg_angle = 2 * numpy.arctan((A + numpy.sqrt(sqrt_arg)) / (B - C))

                            copy_internal_coords(
                                ref_mol, mol, ref_n, ref_ca, ref_cb, ref_cg, n, ca, cb, cg,
                                overwrite_angle = ca_cb_cg_angle, overwrite_dihedral = rot_chi1 / rad_to_deg
                            )

                            # Set coordinates of proline ring hydrogens using internal coordinates
                            copy_internal_coords(
                                ref_mol, mol, ref_c, ref_n, ref_ca, ref_ha, c, n, ca, ha)
                            copy_internal_coords(
                                ref_mol, mol, ref_cg, ref_ca, ref_cb, ref_hb1, cg, ca, cb, hb1)
                            copy_internal_coords(
                                ref_mol, mol, ref_cg, ref_ca, ref_cb, ref_hb2, cg, ca, cb, hb2)
                            copy_internal_coords(
                                ref_mol, mol, ref_cd, ref_cb, ref_cg, ref_hg1, cd, cb, cg, hg1)
                            copy_internal_coords(
                                ref_mol, mol, ref_cd, ref_cb, ref_cg, ref_hg2, cd, cb, cg, hg2)
                            copy_internal_coords(
                                ref_mol, mol, ref_n, ref_cg, ref_cd, ref_hd1, n, cg, cd, hd1)
                            copy_internal_coords(
                                ref_mol, mol, ref_n, ref_cg, ref_cd, ref_hd2, n, cg, cd, hd2)

                            for psi in dihedral_range[central_residue]['psi']:

                                # Set coordinates of next residue by rotating psi
                                oechem.OESetTorsion(mol, n, ca, c, n_next, psi / rad_to_deg)

                                # Set coordinates of proline carbonyl O using internal coordinates of next residue amide
                                # nitrogen and proline C and CA, like an improper torsion
                                copy_internal_coords(
                                    ref_mol, mol, ref_n_next, ref_ca, ref_c, ref_o, n_next, ca, c, o)
 
                                new_mol.NewConf(mol)
                                dihedrals_by_conf_idx[mol_name][i][conf_idx] = {
                                    'phi': float(phi),
                                    'psi': oechem.OEGetTorsion(mol, n, ca, c, n_next) * rad_to_deg,
                                    'omega': oechem.OEGetTorsion(mol, ca_prev, c_prev, n, ca) * rad_to_deg,
                                    'chi1': oechem.OEGetTorsion(mol, n, ca, cb, cg) * rad_to_deg,
                                    'chi2': oechem.OEGetTorsion(mol, ca, cb, cg, cd) * rad_to_deg,
                                    'chi3': oechem.OEGetTorsion(mol, cb, cg, cd, n) * rad_to_deg
                                }
                                conf_idx += 1

                        # Write molecule to SDF
                        if ofs.open(
                            os.path.join('capped_3-mer_conformations', mol_name, f'{mol_name}_rotamer_{i + 1}.sdf')
                        ):

                            oechem.OEWriteMolecule(ofs, new_mol)

                        else:
                            print(f'Error writing {mol_name} rotamer {i + 1}')

                elif mol_rotamers.size > 0:

                    for i in range(max_rotamers[central_residue]):

                        dihedrals_by_conf_idx[mol_name][i] = {}
                        conf_idx = 0

                        # Create copy of OE molecule with no conformers
                        new_mol = oechem.OEMol(mol)
                        new_mol.DeleteConfs()

                        # Set sidechain dihedrals to rotamer values
                        residue_idx = 5 if central_residue == 'CYX' else 2
                        oechem.OESetRotamer(mol_residues[residue_idx], mol_rotamers[i])
                        print(f'{mol_name} rotamer {i:d}')
                        print_dihedrals(mol_residues[residue_idx])

                        # Scan backbone dihedrals
                        for phi in dihedral_range[central_residue]['phi']:

                            oechem.OESetTorsion(mol, c_prev, n, ca, c, phi / rad_to_deg)

                            for psi in dihedral_range[central_residue]['psi']:

                                oechem.OESetTorsion(mol, n, ca, c, n_next, psi / rad_to_deg)
                                new_mol.NewConf(mol)
                                dihedrals_by_conf_idx[mol_name][i][conf_idx] = get_dihedrals(mol_residues[residue_idx])
                                conf_idx += 1

                        # Write molecule to SDF
                        if ofs.open(
                            os.path.join('capped_3-mer_conformations', mol_name, f'{mol_name}_rotamer_{i + 1}.sdf')
                        ):

                            oechem.OEWriteMolecule(ofs, new_mol)

                        else:
                            print(f'Error writing {mol_name} rotamer {i + 1}')

                else:

                    dihedrals_by_conf_idx[mol_name][0] = {}
                    conf_idx = 0

                    # Create copy of OE molecule with no conformers
                    new_mol = oechem.OEMol(mol)
                    new_mol.DeleteConfs()

                    # Scan backbone dihedrals
                    for phi in dihedral_range[central_residue]['phi']:

                        oechem.OESetTorsion(mol, c_prev, n, ca, c, phi / rad_to_deg)

                        for psi in dihedral_range[central_residue]['psi']:

                            oechem.OESetTorsion(mol, n, ca, c, n_next, psi / rad_to_deg)
                            new_mol.NewConf(mol)
                            dihedrals_by_conf_idx[mol_name][0][conf_idx] = get_dihedrals(mol_residues[2])
                            conf_idx += 1

                    # Write molecule to SDF
                    if ofs.open(
                        os.path.join('capped_3-mer_conformations', mol_name, f'{mol_name}_rotamer_1.sdf')
                    ):

                        oechem.OEWriteMolecule(ofs, new_mol)

                    else:
                        print(f'Error writing {mol_name} rotamer 1')

            else:
                print(f'Error reading {mol_name} with SMILES {mol_smiles}')

with open(os.path.join('capped_3-mer_conformations', 'dihedrals_by_conf_idx'), 'w') as out_file:
    json.dump(dihedrals_by_conf_idx, out_file)


Ace-Ala-Ala-Ala-Nme CC(=O)N[C@@H](C)C(=O)N[C@@H](C)C(=O)N[C@@H](C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
Ace-Val-Ala-Val-Nme CC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](C)C(=O)N[C@@H](C(C)C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
Ace-Ala-Arg-Ala-Nme CC(=O)N[C@@H](C)C(=O)N[C@@H](CCC[NH+]=C(N)N)C(=O)N[C@@H](C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0   9.90  -67.0  180.0 -179.0  177.0
 1   6.14  -68.0 -172.0  -64.0  -88.0
 2   6.13  -67.0 -179.0 -176.0  -89.0
 3   5.40  -66.0  179.0   66.0 -172.0
 4   5.30  -68.0  180.0  179.0   91.0
 5   5.19  -66.0  179.0  -67.0  173.0
 6   5.04 -176.0  176.0  179.0  179.0
 7   4.09 -177.0  180.0   63.0   83.0
 8   4.00  -66.0  178.0   65.0   88.0
 9   3.31 -175.0  179.0   67.0 -171.0
10   3.24 -174.0  179.0  -65.0  -85.0
11   3.08  -64.0  -69.0 -175.0  -91.0
12   2.98 -177.0  178.0 -178.0  -90.0
13   2.84 -177.0  177.0  -68.0  171.0
14   2.59  -62.0  -68.0 -177.0 -176.0
15   2.28 -176.0  176.0  176.0   87.0
16   2.20  -63.0  -68.0  -61.0  -86

Ace-Val-Ash-Val-Nme CC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](C(C)C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0  51.48  -69.0  -29.0    0.0    0.0
 1  23.65 -172.0   -2.0    0.0    0.0
 2  16.24   63.0   -2.0    0.0    0.0
 3   8.33 -174.0   74.0    0.0    0.0
Ace-Val-Ash-Val-Nme rotamer 0
-146.245   73.225 -179.344  -69.000  -29.000


Ace-Ala-Asn-Ala-Nme CC(=O)N[C@@H](C)C(=O)N[C@@H](CC(=O)N)C(=O)N[C@@H](C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0  49.01  -70.0  -42.0    0.0    0.0
 1  29.10 -171.0   -1.0    0.0    0.0
 2  14.00   64.0    7.0    0.0    0.0
 3   7.46  -64.0  115.0    0.0    0.0
 4   0.11 -161.0  164.0    0.0    0.0
Ace-Ala-Asn-Ala-Nme rotamer 0
  13.507   73.794 -178.792  -70.000  -42.000


Ace-Val-Asn-Val-Nme CC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](CC(=O)N)C(=O)N[C@@H](C(C)C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0  49.01  -70.0  -42.0    0.0    0.0
 1  29.10 -171.0   -1.0    0.0    0.0
 2  14.00   64.0    7.0    0.0    0.0
 3   7.46  -64.0  115.0    0.0    0.0
 4   0.11 -161.0  164.0    0.0    0.0
Ace-Val-Asn-Val-Nme rotamer 0
-155.169   73.792 -178.791  -70.000  -42.000


Ace-Ala-Asp-Ala-Nme CC(=O)N[C@@H](C)C(=O)N[C@@H](CC(=O)[O-])C(=O)N[C@@H](C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0  51.48  -69.0  -29.0    0.0    0.0
 1  23.65 -172.0   -2.0    0.0    0.0
 2  16.24   63.0   -2.0    0.0    0.0
 3   8.33 -174.0   74.0    0.0    0.0
Ace-Ala-Asp-Ala-Nme rotamer 0
 -92.338   74.160 -178.545  -69.000  -29.000


...

Ace-Val-Trp-Val-Nme CC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](CC1=CNc2c1cccc2)C(=O)N[C@@H](C(C)C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0  33.76  -67.0   97.0    0.0    0.0
 1  18.09 -179.0   65.0    0.0    0.0
 2  15.46 -177.0 -103.0    0.0    0.0
 3  11.73  -68.0   -7.0    0.0    0.0
 4  10.35   62.0  -89.0    0.0    0.0
 5   5.19   60.0   88.0    0.0    0.0
 6   5.13  -68.0  -89.0    0.0    0.0
Ace-Val-Trp-Val-Nme rotamer 0
-133.631   73.445 -178.933  -67.000   97.000


Ace-Ala-Tyr-Ala-Nme CC(=O)N[C@@H](C)C(=O)N[C@@H](Cc1ccc(cc1)O)C(=O)N[C@@H](C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0  48.01  -67.0  -81.0    0.0    0.0
 1  34.53 -178.0   76.0    0.0    0.0
 2  11.57   64.0  -90.0    0.0    0.0
 3   5.55  -68.0  -15.0    0.0    0.0
Ace-Ala-Tyr-Ala-Nme rotamer 0
  79.915   73.448 -178.934  -67.000  -81.000


Ace-Val-Tyr-Val-Nme CC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccc(cc1)O)C(=O)N[C@@H](C(C)C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0  48.01  -67.0  -81.0    0.0    0.0
 1  34.53 -178.0   76.0    0.0    0.0
 2  11.57   64.0  -90.0    0.0    0.0
 3   5.55  -68.0  -15.0    0.0    0.0
Ace-Val-Tyr-Val-Nme rotamer 0
-133.631   73.445 -178.933  -67.000  -81.000


Ace-Ala-Val-Ala-Nme CC(=O)N[C@@H](C)C(=O)N[C@@H](C(C)C)C(=O)N[C@@H](C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0  75.56  176.0    0.0    0.0    0.0
 1  17.94  -62.0    0.0    0.0    0.0
 2   6.20   65.0    0.0    0.0    0.0
Ace-Ala-Val-Ala-Nme rotamer 0
 -52.947   68.618 -178.741  176.000


Ace-Val-Val-Val-Nme CC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](C(C)C)C(=O)N[C@@H](C(C)C)C(=O)NC
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0  75.56  176.0    0.0    0.0    0.0
 1  17.94  -62.0    0.0    0.0    0.0
 2   6.20   65.0    0.0    0.0    0.0
Ace-Val-Val-Val-Nme rotamer 0
  45.929   68.615 -178.740  176.000


# Minimize with OpenFF 2.0.0

To clean up valence terms, minimize dipeptide conformations with harmonic restraints on backbone and sidechain dihedrals. Sometimes the local minimization is unable to satisfy the restraints without distorting the geometry elsewhere. The minimized coordinates are screened for changes in connectivity or for tetravalent atoms that don't have tetrahedral geometry. The connectivity is estimated based on interatomic distances using `guess_connectivity()` from `qcelemental`. Tetrahedral geometry is scored based on the root mean square cosine of the deviation of the six angles centered on the tetravalent atom from the ideal tetrahedral geometry of 109.5 deg.
$$\mathsf{tetrahedral\ score} = \left( \frac {1} {6} \sum_{i=1}^6 \cos^2 \left( \theta_i - 109.5\ \mathsf{deg} \right) \right) ^ {1/2}$$
This score is one if the six angles centered on the tetravalent atom are each exactly 109.5 deg but will decrease toward zero as one or more angles deviates from this tetrahedral ideal.

In [4]:
force_field = ForceField('openff_unconstrained-2.0.0.offxml')
connectivity_tolerance = 1.2
tetrahedral_geometry_tolerance = 0.9

with open(os.path.join('capped_3-mer_conformations', 'dihedrals_by_conf_idx'), 'r') as in_file:
    dihedrals_by_conf_idx = json.load(in_file)

openmm_platform = openmm.Platform.getPlatformByName('Reference')

for central_residue in residue_names:

    for flanking_residue in flanking_residues:

        mol_name = f'Ace-{flanking_residue}-{central_residue}-{flanking_residue}-Nme'.title()

        for i in range(max_rotamers[res_name]):

            rot_idx = i + 1

            # Read OFF molecule
            offmol = Molecule.from_file(
                os.path.join('capped_3-mer_conformations', mol_name, f'{mol_name}_rotamer_{rot_idx}.sdf')
            )
            for j in range(1, len(offmol)):
                offmol[0].add_conformer(offmol[j].conformers[0])
            offmol = offmol[0]

            # Set up OpenMM system
            mapped_smiles = offmol.to_smiles(isomeric = True, mapped = True)
            new_mol = Molecule.from_mapped_smiles(mapped_smiles)
            openmm_system = force_field.create_openmm_system(new_mol.to_topology())

            # Get connectivity to screen conformers after minimization
            expected_connectivity = {tuple(sorted([bond.atom1_index, bond.atom2_index])) for bond in new_mol.bonds}

            # Harmonic restraint for periodic torsion
            # Energy constant is 4184 kJ mol^-1 nm^-2, equivalent to 10 kcal mol^-1 angstrom^-2
            harmonic_restraint = openmm.CustomTorsionForce(
                'k_over_2 * diff * diff; diff = min(dphi, two_pi - dphi); dphi = abs(theta - phi_min)')
            harmonic_restraint.addGlobalParameter('two_pi', 2 * numpy.pi)
            harmonic_restraint.addGlobalParameter('k_over_2', 4184)
            harmonic_restraint.addPerTorsionParameter('phi_min')

            # Set up harmonic restraints for backbone and up to first two sidechain torsions
            dihedral_indices = {}
            for dihedral in dihedral_smarts[central_residue]:

                dihedral_indices[dihedral] = []

                for j, atoms in enumerate(offmol.chemical_environment_matches(dihedral_smarts[central_residue][dihedral])):

                    # For symmetric chemistries, take the lowest index
                    if j > 0:
                        continue

                    oemol = offmol.to_openeye()
                    oe_torsion = oechem.OEGetTorsion(
                        oemol,
                        oemol.GetAtom(oechem.OEHasAtomIdx(atoms[0])),
                        oemol.GetAtom(oechem.OEHasAtomIdx(atoms[1])),
                        oemol.GetAtom(oechem.OEHasAtomIdx(atoms[2])),
                        oemol.GetAtom(oechem.OEHasAtomIdx(atoms[3])),
                    ) * rad_to_deg

                    print(
                        f'{mol_name} {rot_idx:2d} {dihedral:5s} {j:2d} {atoms[0]:2d} {atoms[1]:2d} {atoms[2]:2d} '
                        f'{atoms[3]:2d} {dihedrals_by_conf_idx[mol_name]["0"]["0"][dihedral]:8.3f} {oe_torsion:8.3f}'
                    )

                    torsion_idx = harmonic_restraint.addTorsion(atoms[0], atoms[1], atoms[2], atoms[3], [0.0])
                    dihedral_indices[dihedral].append((torsion_idx, atoms))

            openmm_system.addForce(harmonic_restraint)
            openmm_integrator = openmm.VerletIntegrator(0.001 * unit.femtoseconds)
            openmm_context = openmm.Context(openmm_system, openmm_integrator, openmm_platform)

            # Minimize conformers
            for conf_idx in range(offmol.n_conformers):

                # Set minimum of harmonic restraint for backbone torsions
                for dihedral in dihedral_indices:
                    for torsion_idx, atoms in dihedral_indices[dihedral]:

                        harmonic_restraint.setTorsionParameters(
                            torsion_idx, atoms[0], atoms[1], atoms[2], atoms[3],
                            [dihedrals_by_conf_idx[mol_name][str(i)][str(conf_idx)][dihedral] / rad_to_deg]
                        )

                harmonic_restraint.updateParametersInContext(openmm_context)
                openmm_context.setPositions(offmol.conformers[conf_idx].value_in_unit(unit.nanometers))

                # Get minimized coordinates
                openmm.LocalEnergyMinimizer.minimize(openmm_context)
                minimized_state = openmm_context.getState(getPositions = True)
                minimized_coordinates = unit.quantity.Quantity(
                    numpy.array(minimized_state.getPositions().value_in_unit(unit.nanometers)), unit.nanometers)

                # Check connectivity to screen bad conformers
                tmp_mol = Molecule.from_mapped_smiles(mapped_smiles)
                tmp_mol.add_conformer(minimized_coordinates)
                qcmol = tmp_mol.to_qcschema()
                connectivity = {
                    tuple(sorted(bond))
                    for bond in guess_connectivity(qcmol.symbols, qcmol.geometry, connectivity_tolerance)
                }

                if connectivity == expected_connectivity:

                    # Get a list of atom indices bonded to each atom
                    bond_list = {j: [] for j in range(len(qcmol.atomic_numbers))}
                    for bond in qcmol.connectivity:

                        bond_list[bond[0]].append(bond[1])
                        bond_list[bond[1]].append(bond[0])

                    # Check that tetravalent atoms have tetrahedral geometry
                    good_tetravalent_geometry = True

                    for atom_idx in bond_list:

                        if len(bond_list[atom_idx]) == 4:

                            sum_sq_cos_deviation = 0

                            # Loop over pairs of bonded atoms to get angles
                            for j in range(3):
                                for k in range(i + 1, 4):

                                    # Get square of cosine of deviation from tetrahedral ideal of 109.5 deg
                                    angle = qcmol.measure((bond_list[atom_idx][j], atom_idx, bond_list[atom_idx][k]))
                                    sum_sq_cos_deviation += numpy.square(numpy.cos((angle - 109.5) / rad_to_deg))

                            # Screen conformers that deviate from tetrahedral geometry
                            tetrahedral_score = numpy.sqrt(sum_sq_cos_deviation / 6)

                            if tetrahedral_score < tetrahedral_geometry_tolerance:

                                good_tetravalent_geometry = False

                                print(
                                    f'{mol_name} rotamer {rot_idx} conformer {conf_idx} with phi '
                                    f'{dihedrals_by_conf_idx[mol_name][str(i)][str(conf_idx)]["phi"]:6.1f} and psi '
                                    f'{dihedrals_by_conf_idx[mol_name][str(i)][str(conf_idx)]["psi"]:6.1f} has a '
                                    f'tetravalent atom {qcmol.symbols[atom_idx]}{atom_idx} with non-tetrahedral geometry '
                                    f'(score {tetrahedral_score:5.3f}) and was skipped.'
                                )

                    # If good connectivity and good tetravalent geometry, retain this minimized conformer
                    if good_tetravalent_geometry:
                        new_mol.add_conformer(minimized_coordinates)

                else:

                    print(
                        f'{mol_name} rotamer {rot_idx} conformer {conf_idx} with phi '
                        f'{dihedrals_by_conf_idx[mol_name][str(i)][str(conf_idx)]["phi"]:6.1f} and psi '
                        f'{dihedrals_by_conf_idx[mol_name][str(i)][str(conf_idx)]["psi"]:6.1f} changed connectivity and '
                        'was skipped.'
                    )

            # Align conformers to first conformer using heavy atoms of acetyl group and N and CA of amino acid
            oemol = new_mol.to_openeye()
            ref_oemol = oechem.OEGraphMol(oemol.GetConf(oechem.OEHasConfIdx(0)))

            # Center reference molecule
            oechem.OECenter(ref_oemol)

            # Substructure search for methyl carbon and carbonyl atoms of acetyl group and N and CA of amino acid
            ace_n_ca_subsearch = oechem.OESubSearch()
            ace_n_ca_subsearch.Init('[#6X4H3]-[#6X3](=O)-[#7X3]-[#6X4]')

            # Create atom match pairs between target and reference molecules
            alignment_match = oechem.OEMatch()
            oesubsearch_match_unique = True
            for ref_match in ace_n_ca_subsearch.Match(ref_oemol, oesubsearch_match_unique):
                for target_match in ace_n_ca_subsearch.Match(oemol, oesubsearch_match_unique):
                    for ref_atoms, target_atoms in zip(ref_match.GetAtoms(), target_match.GetAtoms()):
                        alignment_match.AddPair(ref_atoms.target, target_atoms.target)

            # Set up OpenEye RMSD alignment
            oermsd_overlay = True
            N_confs = oemol.GetMaxConfIdx()
            rmsd_vector = oechem.OEDoubleArray(N_confs)
            rotation_matrix = oechem.OEDoubleArray(9 * N_confs)
            translation_vector = oechem.OEDoubleArray(3 * N_confs)

            # Compute rotation matrix and translation vector of alignment
            oechem.OERMSD(
                ref_oemol, oemol, rmsd_vector, alignment_match, oermsd_overlay, rotation_matrix, translation_vector
            )

            # Apply rotation then translation to align target molecule to reference molecule
            oechem.OERotate(oemol, rotation_matrix)
            oechem.OETranslate(oemol, translation_vector)

            # Write molecule with minimized and aligned conformers to SDF
            with oechem.oemolostream() as ofs:

                if ofs.open(
                    os.path.join('capped_3-mer_conformations', mol_name, f'{mol_name}_min_rotamer_{rot_idx}.sdf')
                ):

                    oechem.OEWriteMolecule(ofs, oemol)

                else:
                    print(f'Error writing {mol_name} rotamer {rot_idx}')


Ace-Ala-Ala-Ala-Nme  1 phi    0  8 16 17 18 -180.000  179.997
Ace-Ala-Ala-Ala-Nme  1 psi    0 16 17 18 26  180.000  179.999


/home/ccavende/anaconda3/envs/openff-qca/lib/python3.9/site-packages/qcelemental/util/misc.py:254: RuntimeWarning: invalid value encountered in arccos
  angle = np.pi - np.arccos(cosine_angle)


Ace-Ala-Ala-Ala-Nme rotamer 1 conformer 322 with phi   15.0 and psi  -30.0 changed connectivity and was skipped.
Ace-Val-Ala-Val-Nme  1 phi    0  8 22 23 24 -180.000  179.999
Ace-Val-Ala-Val-Nme  1 psi    0 22 23 24 32  180.000 -179.997
Ace-Val-Ala-Val-Nme rotamer 1 conformer 346 with phi   30.0 and psi  -30.0 changed connectivity and was skipped.
Ace-Val-Ala-Val-Nme rotamer 1 conformer 347 with phi   30.0 and psi  -15.0 changed connectivity and was skipped.
Ace-Ala-Arg-Ala-Nme  1 phi    0  8 16 17 18  180.000 -179.999
Ace-Ala-Arg-Ala-Nme  1 psi    0 16 17 18 40 -180.000 -180.000
Ace-Ala-Arg-Ala-Nme  1 chi1   0 16 17 20 21  -67.000  -66.999
Ace-Ala-Arg-Ala-Nme  1 chi2   0 17 20 21 22 -180.000 -179.998
Ace-Ala-Arg-Ala-Nme rotamer 1 conformer 322 with phi   15.0 and psi  -30.0 changed connectivity and was skipped.
Ace-Ala-Arg-Ala-Nme rotamer 1 conformer 541 with phi  150.0 and psi   15.0 changed connectivity and was skipped.
Ace-Val-Arg-Val-Nme  1 phi    0  8 22 23 24  180.000  179.996
A

Ace-Ala-Hid-Ala-Nme  1 chi1   0 16 17 20 21  -64.000  -64.004
Ace-Ala-Hid-Ala-Nme  1 chi2   0 17 20 21 22  -75.000  -75.000
Ace-Val-Hid-Val-Nme  1 phi    0  8 22 23 24  180.000 -179.996
Ace-Val-Hid-Val-Nme  1 psi    0 22 23 24 39 -180.000  179.998
Ace-Val-Hid-Val-Nme  1 chi1   0 22 23 26 27  -64.000  -63.997
Ace-Val-Hid-Val-Nme  1 chi2   0 23 26 27 28  -75.000  -75.000
Ace-Val-Hid-Val-Nme rotamer 1 conformer 300 with phi   -0.0 and psi    0.0 changed connectivity and was skipped.
Ace-Val-Hid-Val-Nme rotamer 1 conformer 323 with phi   15.0 and psi  -15.0 changed connectivity and was skipped.
Ace-Ala-Hie-Ala-Nme  1 phi    0  8 16 17 18 -180.000  179.997
Ace-Ala-Hie-Ala-Nme  1 psi    0 16 17 18 33 -180.000 -179.999
Ace-Ala-Hie-Ala-Nme  1 chi1   0 16 17 20 21  -64.000  -64.001
Ace-Ala-Hie-Ala-Nme  1 chi2   0 17 20 21 22  -75.000  -74.997
Ace-Val-Hie-Val-Nme  1 phi    0  8 22 23 24 -180.000 -179.996
Ace-Val-Hie-Val-Nme  1 psi    0 22 23 24 39 -180.000 -179.996
Ace-Val-Hie-Val-Nme  1 chi1   

Ace-Val-Cispro-Val-Nme rotamer 1 conformer 261 with phi  -30.0 and psi  135.0 changed connectivity and was skipped.
Ace-Val-Cispro-Val-Nme rotamer 1 conformer 294 with phi    0.0 and psi  -90.0 changed connectivity and was skipped.
Ace-Val-Cispro-Val-Nme rotamer 1 conformer 317 with phi   15.0 and psi -105.0 changed connectivity and was skipped.
Ace-Val-Cispro-Val-Nme rotamer 1 conformer 318 with phi   15.0 and psi  -90.0 changed connectivity and was skipped.
Ace-Val-Cispro-Val-Nme rotamer 1 conformer 341 with phi   30.0 and psi -105.0 changed connectivity and was skipped.
Ace-Val-Cispro-Val-Nme rotamer 1 conformer 342 with phi   30.0 and psi  -90.0 changed connectivity and was skipped.
Ace-Val-Cispro-Val-Nme rotamer 1 conformer 355 with phi   30.0 and psi  105.0 changed connectivity and was skipped.
Ace-Val-Cispro-Val-Nme rotamer 1 conformer 367 with phi   45.0 and psi  -75.0 changed connectivity and was skipped.
Ace-Val-Cispro-Val-Nme rotamer 1 conformer 377 with phi   45.0 and psi  

# Setup 2-D TorsionDrive

Setup 2-D TorsionDrives on chi1 and chi2.

In [5]:
# Get software provenance
factory = TorsiondriveDatasetFactory()
provenance = factory.provenance(GLOBAL_TOOLKIT_REGISTRY)

# Initialize TorsionDrive dataset
dataset = TorsiondriveDataset(
    dataset_name = 'OpenFF Protein Capped 3-mer Backbones v1.0',
    dataset_tagline = 'Capped 3-mer 2-D TorsionDrives on backbone dihedrals for central residue',
    description = 'TorsionDrives on phi and psi for capped 3-mers of Ace-Y-X-Y-Nme with Y = {Ala, Val} and X = 27 '
        'canonical amino acids (Ash, Cyx, Glh, Hid, Hip, and Lyn and Pro with omega both cis and trans)',
    provenance = provenance
)
dataset.metadata.submitter = 'chapincavender'
dataset.metadata.long_description_url = (
    'https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/'
    '2022-05-30-OpenFF-Protein-Capped-3-mer-Backbones'
)

# Add molecules with constraints on non-driven backbone torsions to dataset
for central_residue in residue_names:

    res_rotamers = get_sorted_rotamers(central_residue)

    for flanking_residue in flanking_residues:

        mol_name = f'Ace-{flanking_residue}-{central_residue}-{flanking_residue}-Nme'.title()

        for i in range(max_rotamers[central_residue]):

            rot_idx = i + 1

            # Read molecule with minimized conformers
            offmol = Molecule.from_file(
                os.path.join('capped_3-mer_conformations', mol_name, f'{mol_name}_min_rotamer_{rot_idx}.sdf')
            )
            for j in range(1, len(offmol)):
                offmol[0].add_conformer(offmol[j].conformers[0])
            offmol = offmol[0]

            print(
                f'{mol_name} rotamer {rot_idx:d} of {max_rotamers[central_residue]:d} ({offmol.n_conformers:d} conformers)'
            )

            # Molecule metadata
            mol_index = f'{mol_name}-rotamer-{rot_idx:d}'

            # Indices and scan range for driven torsions (phi and psi)
            phi_indices = offmol.chemical_environment_matches(dihedral_smarts[central_residue]['phi'])[0]
            psi_indices = offmol.chemical_environment_matches(dihedral_smarts[central_residue]['psi'])[0]        
            phi_range = (dihedral_range[central_residue]['phi'][0], dihedral_range[central_residue]['phi'][-1])    
            psi_range = (dihedral_range[central_residue]['psi'][0], dihedral_range[central_residue]['psi'][-1])
            phi_spacing = dihedral_spacing[central_residue]['phi']
            psi_spacing = dihedral_spacing[central_residue]['psi']

            # Add molecule to dataset
            dataset.add_molecule(
                index = mol_index,
                molecule = offmol,
                dihedrals = [phi_indices, psi_indices],
                keywords = {'dihedral_ranges': [phi_range, psi_range], 'grid_spacing': [phi_spacing, psi_spacing]}
            )

            # Add constraints for non-driven torsions (chi1, chi2, and for proline omega)
            for dihedral in dihedral_smarts[central_residue]:

                for j, atoms in enumerate(offmol.chemical_environment_matches(dihedral_smarts[central_residue][dihedral])):

                    # For symmetric chemistries, take the lowest index
                    if j > 0:
                        continue

                    if dihedral == 'chi1':
                        value = res_rotamers[i].GetChi1()
                    elif dihedral == 'chi2':
                        value = res_rotamers[i].GetChi2()
                    elif dihedral == 'omega':
                        value = 0 if central_residue == 'cisPRO' else 180
                    else:
                        continue

                    print(
                        f'{mol_name} {rot_idx:2d} {dihedral:5s} {j:2d} {atoms[0]:2d} {atoms[1]:2d} {atoms[2]:2d} '
                        f'{atoms[3]:2d} {value:8.3f}'
                    )

                    dataset.dataset[mol_index].add_constraint(
                        constraint = 'set', constraint_type = 'dihedral', indices = atoms, value = value)


#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
Ace-Ala-Ala-Ala-Nme rotamer 1 of 1 (575 conformers)
Ace-Val-Ala-Val-Nme rotamer 1 of 1 (574 conformers)
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0   9.90  -67.0  180.0 -179.0  177.0
 1   6.14  -68.0 -172.0  -64.0  -88.0
 2   6.13  -67.0 -179.0 -176.0  -89.0
 3   5.40  -66.0  179.0   66.0 -172.0
 4   5.30  -68.0  180.0  179.0   91.0
 5   5.19  -66.0  179.0  -67.0  173.0
 6   5.04 -176.0  176.0  179.0  179.0
 7   4.09 -177.0  180.0   63.0   83.0
 8   4.00  -66.0  178.0   65.0   88.0
 9   3.31 -175.0  179.0   67.0 -171.0
10   3.24 -174.0  179.0  -65.0  -85.0
11   3.08  -64.0  -69.0 -175.0  -91.0
12   2.98 -177.0  178.0 -178.0  -90.0
13   2.84 -177.0  177.0  -68.0  171.0
14   2.59  -62.0  -68.0 -177.0 -176.0
15   2.28 -176.0  176.0  176.0   87.0
16   2.20  -63.0  -68.0  -61.0  -86.0
17   2.05  -62.0  -66.0  -64.0  163.0
18   1.78 -179.0   66.0  178.0  171.0
19   1.77   65.0 -177.0 -180.0  179.0
20   1.76   65.0  179.0  177.0   88.0
21   1.68  -68.0 -177.

Ace-Ala-Hie-Ala-Nme rotamer 1 of 1 (576 conformers)
Ace-Ala-Hie-Ala-Nme  1 chi1   0 16 17 20 21  -64.000
Ace-Ala-Hie-Ala-Nme  1 chi2   0 17 20 21 22  -75.000
Ace-Val-Hie-Val-Nme rotamer 1 of 1 (572 conformers)
Ace-Val-Hie-Val-Nme  1 chi1   0 22 23 26 27  -64.000
Ace-Val-Hie-Val-Nme  1 chi2   0 23 26 27 28  -75.000
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0  31.73  -64.0  -75.0    0.0    0.0
 1  17.01 -178.0   74.0    0.0    0.0
 2  13.14  -66.0   88.0    0.0    0.0
 3  11.93 -173.0  -87.0    0.0    0.0
 4   9.05  -68.0  171.0    0.0    0.0
 5   7.39   65.0  -81.0    0.0    0.0
 6   5.01   62.0   87.0    0.0    0.0
 7   4.47 -173.0 -167.0    0.0    0.0
Ace-Ala-Hip-Ala-Nme rotamer 1 of 1 (576 conformers)
Ace-Ala-Hip-Ala-Nme  1 chi1   0 16 17 20 21  -64.000
Ace-Ala-Hip-Ala-Nme  1 chi2   0 17 20 21 22  -75.000
Ace-Val-Hip-Val-Nme rotamer 1 of 1 (573 conformers)
Ace-Val-Hip-Val-Nme  1 chi1   0 22 23 26 27  -64.000
Ace-Val-Hip-Val-Nme  1 chi2   0 23 26 27 28  -75.000
#   Prob   Chi 1  Chi 2  C

Ace-Ala-Cispro-Ala-Nme rotamer 1 of 1 (399 conformers)
Ace-Ala-Cispro-Ala-Nme  1 chi1   0 16 17 20 21  -25.000
Ace-Ala-Cispro-Ala-Nme  1 chi2   0 17 20 21 22   36.000
Ace-Ala-Cispro-Ala-Nme  1 omega  0  7  8 16 17    0.000
Ace-Val-Cispro-Val-Nme rotamer 1 of 1 (389 conformers)
Ace-Val-Cispro-Val-Nme  1 chi1   0 22 23 26 27  -25.000
Ace-Val-Cispro-Val-Nme  1 chi2   0 23 26 27 28   36.000
Ace-Val-Cispro-Val-Nme  1 omega  0  7  8 22 23    0.000
#   Prob   Chi 1  Chi 2  Chi 3  Chi 4
 0  50.62  -25.0   36.0    0.0    0.0
 1  49.05   27.0  -34.0    0.0    0.0
Ace-Ala-Transpro-Ala-Nme rotamer 1 of 1 (406 conformers)
Ace-Ala-Transpro-Ala-Nme  1 chi1   0 16 17 20 21  -25.000
Ace-Ala-Transpro-Ala-Nme  1 chi2   0 17 20 21 22   36.000
Ace-Ala-Transpro-Ala-Nme  1 omega  0  7  8 16 17  180.000
Ace-Val-Transpro-Val-Nme rotamer 1 of 1 (408 conformers)
Ace-Val-Transpro-Val-Nme  1 chi1   0 22 23 26 27  -25.000
Ace-Val-Transpro-Val-Nme  1 chi2   0 23 26 27 28   36.000
Ace-Val-Transpro-Val-Nme  1 omega  0

Describe and export the dataset

In [7]:
confs = numpy.array([mol.n_conformers for mol in dataset.molecules])
molecular_weights = numpy.array([oechem.OECalculateMolecularWeight(mol.to_openeye()) for mol in dataset.molecules])
unique_formal_charges = numpy.unique([mol.total_charge / mol.total_charge.unit for mol in dataset.molecules])

print(f'Number of unique molecules        {dataset.n_molecules:d}')
print(f'Number of filtered molecules      {dataset.n_filtered:d}')
print(f'Number of torsion drives          {dataset.n_records:d}')
print(f'Number of conformers min mean max {confs.min():3d} {confs.mean():6.2f} {confs.max():3d}')
print(
    f'Molecular weight min mean max     {molecular_weights.min():6.2f} {molecular_weights.mean():6.2f} '
    f'{molecular_weights.max():6.2f}'
)
print(f'Charges                          ', sorted(unique_formal_charges))

print(dataset.metadata.dict())

for spec, obj in dataset.qc_specifications.items():
    print("Spec:", spec)
    print(obj.dict())


Number of unique molecules        52
Number of filtered molecules      0
Number of torsion drives          54
Number of conformers min mean max 389 561.94 576
Molecular weight min mean max     272.30 369.22 548.72
Charges                           [-1.0, 0.0, 1.0]
{'submitter': 'chapincavender', 'creation_date': datetime.date(2022, 5, 30), 'collection_type': 'TorsionDriveDataset', 'dataset_name': 'OpenFF Protein Capped 3-mer Backbones v1.0', 'short_description': 'Capped 3-mer 2-D TorsionDrives on backbone dihedrals for central residue', 'long_description_url': HttpUrl('https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2022-05-30-OpenFF-Protein-Capped-3-mer-Backbones', scheme='https', host='github.com', tld='com', host_type='domain', path='/openforcefield/qca-dataset-submission/tree/master/submissions/2022-05-30-OpenFF-Protein-Capped-3-mer-Backbones'), 'long_description': 'TorsionDrives on phi and psi for capped 3-mers of Ace-Y-X-Y-Nme with Y = {Ala, Val} 

In [8]:
dataset.export_dataset('dataset.json.bz2')

In [9]:
dataset.molecules_to_file('dataset.smi', 'smi')
dataset.visualize('dataset.pdf')